In [1]:
# 参考https://zhuanlan.zhihu.com/p/97079690
from pdfminer.pdfparser import  PDFParser,PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal,LAParams
from pdfminer.pdfinterp import PDFTextExtractionNotAllowed

#检查整个字符串是否包含中文
def include_chinese(string):
    for ch in string:
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False

def parse(pdf_path,txt_path):
    '''解析PDF文本，并保存到TXT文件中'''
    fp = open(pdf_path,'rb')
    #用文件对象创建一个PDF文档分析器
    parser = PDFParser(fp)
    #创建一个PDF文档
    doc = PDFDocument()
    #连接分析器，与文档对象
    parser.set_document(doc)
    doc.set_parser(parser)

    #提供初始化密码，如果没有密码，就创建一个空的字符串
    doc.initialize()
 
    #检测文档是否提供txt转换，不提供就忽略
    if not doc.is_extractable:
        raise PDFTextExtractionNotAllowed
    else:
        #创建PDF，资源管理器，来共享资源
        rsrcmgr = PDFResourceManager()
        #创建一个PDF设备对象
        laparams = LAParams()
        device = PDFPageAggregator(rsrcmgr,laparams=laparams)
        #创建一个PDF解释器对象
        interpreter = PDFPageInterpreter(rsrcmgr,device)
 
        #循环遍历列表，每次处理一个page内容
        # doc.get_pages() 获取page列表
        for page in doc.get_pages():
            interpreter.process_page(page)
            #接受该页面的LTPage对象
            layout = device.get_result()
            # 这里layout是一个LTPage对象 里面存放着 这个page解析出的各种对象
            # 一般包括LTTextBox, LTFigure, LTImage, LTTextBoxHorizontal 等等
            # 想要获取文本就获得对象的text属性，
            for x in layout:
                if(isinstance(x,LTTextBoxHorizontal)):
                    with open(txt_path,'a') as f:
                        results = x.get_text()

                        # 排除PDF文档里的无关项后，输出到.txt文件
                        useless = ['SE', 'HD', 'BY', 'WC', 'PD', 'SN', 'SC', 'ED', 'PG', 'VOL', 'LA', 'CY', 'LP','TD']
                        lines = results.splitlines()
                        uselessFlag = 0
                        for line in lines:
                            '''
                            #无用信息通常从'gfr :'开始，到'德语'结束，故这段内容不输出即可
                            if uselessFlag == 1:
                                if '德语' in line:
                                    uselessFlag = 0
                                    print('\n\n')
                                continue
                            if len(line) == 2:
                                uselessFlag = 1
                                continue
                            '''
                            if include_chinese(line):
                                continue
                            if line in useless:
                                continue
                            else:
                                f.write(line + "\n")
 
def main():
    pdf_path = 'afterPandemic.pdf'
    txt_path = 'newMyCorpus_after.txt'
    parse(pdf_path,txt_path)

main()